In [5]:
import sys
sys.path.append('/home/ec2-user/CpGPython/code/')
import pandas as pd
import numpy as np
from hyperopt import fmin,tpe,hp, STATUS_OK,Trials 
import deep_network_estimator as dne
from sklearn.model_selection import cross_validate
import math
from sklearn.externals import joblib
import Logger
import argparse
import xgbooster

In [23]:
from importlib import reload
reload(dne)

<module 'deep_network_estimator' from '/home/ec2-user/CpGPython/code/deep_network_estimator.py'>

In [2]:
def cal_opt_score(estimator):
    results = cross_validate(estimator,train_x,train_label,scoring=scoring,cv=cv,return_train_score=False,fit_params={'sample_weight':sample_weights_train})
    if 'f1_macro' in scoring:
        score = results['test_f1_macro'].mean()
    elif 'f1' in  scoring:
        score = results['test_f1'].mean()
    if math.isnan(score) or math.isinf(score):
        score = - np.Infinity
    return score
    
def dnn_loss(params):
    global estimators
    dnn_estimator = dne.tensor_DNN(**params)
    score = cal_opt_score(dnn_estimator)
    estimators.extend([dnn_estimator])   
    return {'loss':-score,'status':STATUS_OK}

In [98]:
xgb_param = {'learning_rate':hp.choice('learning_rate',[0.1]),'max_depth': 3+hp.randint('max_depth',15),'n_estimators':500+hp.randint('n_estimators',3000),'reg_lambda': hp.uniform('reg_lambda',1,100),'gamma': hp.uniform('gamma',1,30)}

parser = argparse.ArgumentParser(description='Logistic Regression')
parser.add_argument('-f',required=False,default='normal',help='feature set',dest='features',metavar='normal/autoencoder')
parser.add_argument('-i',required=False,default=30,help='max iteration',dest='maxiter',metavar='30',type=int)
parser.add_argument('-c',required=False,default=3,help='cv number',dest='cv',metavar='3',type=int)
args = parser.parse_args()
feature_dataset = args.features
cv = args.cv
max_iter = args.maxiter 

In [3]:
feature_dataset = 'normal'
cv = 2
max_iter = 2
home='/home/ec2-user/CpGPython/'
log_dir = home+'logs/'
model_dir = home+'models/'
logger = Logger.Logger(log_dir,False).get_logger()
if feature_dataset == 'normal':
##features selecetd by traditional methods
    with pd.HDFStore(home+'data/selected_features','r') as h5s:
        train_x =h5s['train_x'] 
        train_label = h5s['train_label'] 
        test_x = h5s['test_x'] 
        test_label = h5s['test_label']   
        sample_weights_train = h5s['sample_weights_train'] 
        sample_weights_test = h5s['sample_weights_test'] 
    logger.info('Features used in training are from traditional feature selection')
##features selected by sparse autoencoder
elif feature_dataset == 'autoencoder':
    with pd.HDFStore(home+'data/new_features','r') as h5s:
        train_x =h5s['train_x'] 
        train_label = h5s['train_label'] 
        test_x = h5s['test_x'] 
        test_label = h5s['test_label']
        sample_weights_train = h5s['sample_weights_train'] 
        sample_weights_test = h5s['sample_weights_test']
    logger.info('Features used in training are from sparse autoencoder')
train_x = pd.DataFrame(train_x)
train_label = pd.Series(train_label)
sample_weights_train = pd.Series(sample_weights_train)

labels = train_label.unique()
class_num = len(labels)
feature_num = train_x.shape[1]
dnn_param = {'batch_normalization': hp.choice('batch_normalization',[True]),
             'l2_reg': hp.uniform('l2_reg',0.001,0.05),                            
             'drop_out':hp.uniform('drop_out',0.1,0.5),
             #'weight_factor':hp.uniform('weight_factor',1,2),
             'steps':200+hp.randint('steps',2000),
             'batch_size':hp.choice('batch_size',[30]),
             'scoring':hp.choice('scoring',['precision']),
             'n_classes':hp.choice('n_classes',[class_num]),
             'hidden_layers':hp.choice('hidden_layers',[[int(feature_num*5),int(feature_num*3),int(feature_num*1)],[int(feature_num*4),int(feature_num*3),int(feature_num*2),int(feature_num*1)],[int(feature_num*3),int(feature_num*2.5),int(feature_num*2),int(feature_num*1.5),int(feature_num*1)],[int(feature_num*6),int(feature_num*3)]])
             }

if class_num <= 2:
    scoring = ['precision','recall','f1','roc_auc','neg_log_loss']
else:
    scoring = ['precision_macro','recall_macro','f1_macro','neg_log_loss']


In [5]:
trial = Trials()
estimators = []
best = fmin(dnn_loss,dnn_param,algo=tpe.suggest,max_evals=max_iter,trials=trial)
best_ix = np.argmin(trial.losses())
best_estimator = estimators[best_ix]
best_params = best_estimator.get_params() 


Tensor("IteratorGetNext:37", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:37", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:37", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:37", shape=(?,), dtype=float64)


['/home/ec2-user/CpGPython/models/tensor_DNN.pkl']

In [2]:
def cal_opt_score(estimator):
    results = cross_validate(estimator,train_x,train_label,scoring=scoring,cv=cv,return_train_score=False,fit_params={'sample_weight':sample_weights_train})
    if 'f1_macro' in scoring:
        score = results['test_f1_macro'].mean()
    elif 'f1' in  scoring:
        score = results['test_f1'].mean()
    if math.isnan(score) or math.isinf(score):
        score = - np.Infinity
    return score

def xgb_loss(params):
    global estimators
    xgb_estimator = xgbooster.xgbooster(**params)
    score = cal_opt_score(xgb_estimator)
    estimators.extend([xgb_estimator])    
    return {'loss':-score,'status':STATUS_OK}

In [3]:
xgb_param = {'learning_rate':hp.choice('learning_rate',[0.1]),'max_depth': 3+hp.randint('max_depth',15),'n_estimators':500+hp.randint('n_estimators',3000),'reg_lambda': hp.uniform('reg_lambda',1,100),'gamma': hp.uniform('gamma',1,30)}
feature_dataset = 'normal'
cv = 2
max_iter = 2
home='/home/ec2-user/CpGPython/'
log_dir = home+'logs/'
model_dir = home+'models/'
logger = Logger.Logger(log_dir,False).get_logger()
if feature_dataset == 'normal':
##features selecetd by traditional methods
    with pd.HDFStore(home+'data/selected_features','r') as h5s:
        train_x =h5s['train_x'] 
        train_label = h5s['train_label'] 
        test_x = h5s['test_x'] 
        test_label = h5s['test_label']   
        sample_weights_train = h5s['sample_weights_train'] 
        sample_weights_test = h5s['sample_weights_test'] 
    logger.info('Features used in training are from traditional feature selection')
##features selected by sparse autoencoder
elif feature_dataset == 'autoencoder':
    with pd.HDFStore(home+'data/new_features','r') as h5s:
        train_x =h5s['train_x'] 
        train_label = h5s['train_label'] 
        test_x = h5s['test_x'] 
        test_label = h5s['test_label']
        sample_weights_train = h5s['sample_weights_train'] 
        sample_weights_test = h5s['sample_weights_test']
    logger.info('Features used in training are from sparse autoencoder')
train_x = pd.DataFrame(train_x)
train_label = pd.Series(train_label)
sample_weights_train = pd.Series(sample_weights_train)

In [6]:
labels = train_label.unique()
class_num = len(labels)

if class_num <= 2:
    scoring = ['precision','recall','f1','roc_auc','neg_log_loss']
else:
    scoring = ['precision_macro','recall_macro','f1_macro','neg_log_loss']

trial = Trials()
estimators = []
best = fmin(xgb_loss,xgb_param,algo=tpe.suggest,max_evals=max_iter,trials=trial) 
best_ix = np.argmin(trial.losses())
best_estimator = estimators[best_ix]
best_params = best_estimator.get_params() 

{'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bytree': 1, 'gamma': 11.929395949493868, 'learning_rate': 0.1, 'max_delta_step': 0, 'max_depth': 12, 'min_child_weight': 1, 'missing': None, 'n_estimators': 3080, 'n_jobs': 1, 'nthread': None, 'objective': 'binary:logistic', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 14.06628287299413, 'scale_pos_weight': 1, 'seed': None, 'silent': True, 'subsample': 1}


/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


{'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bytree': 1, 'gamma': 11.929395949493868, 'learning_rate': 0.1, 'max_delta_step': 0, 'max_depth': 12, 'min_child_weight': 1, 'missing': None, 'n_estimators': 3080, 'n_jobs': 1, 'nthread': None, 'objective': 'binary:logistic', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 14.06628287299413, 'scale_pos_weight': 1, 'seed': None, 'silent': True, 'subsample': 1}


/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


{'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bytree': 1, 'gamma': 6.064439877953024, 'learning_rate': 0.1, 'max_delta_step': 0, 'max_depth': 14, 'min_child_weight': 1, 'missing': None, 'n_estimators': 2271, 'n_jobs': 1, 'nthread': None, 'objective': 'binary:logistic', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 37.198308117201016, 'scale_pos_weight': 1, 'seed': None, 'silent': True, 'subsample': 1}


/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


{'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bytree': 1, 'gamma': 6.064439877953024, 'learning_rate': 0.1, 'max_delta_step': 0, 'max_depth': 14, 'min_child_weight': 1, 'missing': None, 'n_estimators': 2271, 'n_jobs': 1, 'nthread': None, 'objective': 'binary:logistic', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 37.198308117201016, 'scale_pos_weight': 1, 'seed': None, 'silent': True, 'subsample': 1}


/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [7]:
best_params

{'gamma': 6.064439877953024,
 'learning_rate': 0.1,
 'max_depth': 14,
 'n_estimators': 2271,
 'reg_lambda': 37.198308117201016}

In [4]:
train_x

,Fetal_Placenta-H3K4me1,K562-XRCC4,CD14_Primary_Cells-H3K4me1,H9-H4K8ac,CADD_max_raw,CADD_avg_phred,Breast_Luminal_Epithelial_Cells-H3K36me3,Chorion,NHDF-Ad-H3K9me3,Monocytes-CD14+_RO01746-H3K4me1,...,CD20+-H3K4me2,Medullo,Neurosphere_Cultured_Cells_Cortex_Derived-H3K36me3,Penis_Foreskin_Fibroblast_Primary_Cells-H3K9me3,IMR901.0,Adipose_Nuclei-H3K4me1,Psoas_Muscle-H3K27ac,H1-hESC-BRCA1,Right_Atrium-H3K27ac,genocanyon_score
0,0.019608,0.125,0.000000,0.09375,0.085176,0.089447,0.451220,0.004053,0.130435,0.000000,...,0.000000,0.203390,0.336735,0.000000,0.025384,0.113445,0.000000,0.222222,0.198953,0.995779
1,0.313725,0.250,0.056872,0.00000,0.204069,0.415645,0.000000,0.062817,0.391304,0.198020,...,0.060345,0.033898,0.040816,0.153846,0.111556,0.308824,0.725000,0.333333,0.743455,0.871420
2,0.274510,0.250,0.535545,0.12500,0.083295,0.078516,0.000000,0.014184,0.043478,0.594059,...,0.008621,0.305085,0.010204,0.048077,0.102204,0.123950,0.000000,0.111111,0.000000,0.971669
3,0.029412,0.250,0.000000,0.03125,0.102006,0.118020,0.134146,0.019250,0.000000,0.000000,...,0.000000,0.610169,0.000000,0.000000,0.204409,0.033613,0.000000,0.333333,0.000000,0.999034
4,0.019608,0.125,0.000000,0.21875,0.189200,0.348709,0.792683,0.059777,0.000000,0.009901,...,0.006897,0.033898,0.663265,0.067308,0.056112,0.012605,0.000000,0.000000,0.036649,0.557168
5,0.245098,0.000,0.170616,0.06250,0.109536,0.174964,0.000000,0.022290,0.000000,0.178218,...,0.265517,0.237288,0.000000,0.038462,0.073480,0.367647,0.017857,0.388889,0.010471,0.259607
6,0.058824,0.000,0.000000,0.18750,0.080042,0.066717,0.536585,0.051672,0.260870,0.000000,...,0.077586,0.101695,0.408163,0.028846,0.063460,0.220588,0.003571,0.000000,0.000000,0.507490
7,0.147059,0.125,0.000000,0.25000,0.078334,0.048645,0.097561,0.014184,0.000000,0.029703,...,0.000000,0.135593,0.000000,0.000000,0.043420,0.220588,0.007143,0.000000,0.005236,0.994688
8,0.000000,0.375,0.018957,0.15625,0.069203,0.031538,0.012195,0.038501,0.391304,0.000000,...,0.000000,0.135593,0.306122,0.461538,0.052104,0.037815,0.000000,0.000000,0.000000,0.087926
9,0.000000,0.000,0.000000,0.12500,0.099088,0.146997,0.365854,0.028369,0.000000,0.000000,...,0.000000,0.118644,0.744898,0.000000,0.062124,0.018908,0.000000,0.222222,0.000000,0.268231


In [21]:
command_line = '{}code/HyperoptModels/run-opts.sh {} {} {}'.format(home,feature_dataset,max_iter,cv)
args = shlex.split(command_line)

In [25]:
subprocess.call([home+'code/HyperoptModels/run-opts.sh',str(feature_dataset),str(max_iter),str(cv)])

0

In [20]:
import subprocess,shlex

In [22]:
p = subprocess.Popen(args)

In [24]:
p.wait()

0

In [3]:
feature_dataset = 'normal'
cv = 3
max_iter = 10
home='/home/ec2-user/CpGPython/'
log_dir = home+'logs/'
model_dir = home+'models/'
logger = Logger.Logger(log_dir,False).get_logger()
if feature_dataset == 'normal':
##features selecetd by traditional methods
    with pd.HDFStore(home+'data/selected_features','r') as h5s:
        train_x =h5s['train_x'] 
        train_label = h5s['train_label'] 
        test_x = h5s['test_x'] 
        test_label = h5s['test_label']   
        sample_weights_train = h5s['sample_weights_train'] 
        sample_weights_test = h5s['sample_weights_test'] 
    logger.info('Features used in training are from traditional feature selection')
##features selected by sparse autoencoder
elif feature_dataset == 'autoencoder':
    with pd.HDFStore(home+'data/new_features','r') as h5s:
        train_x =h5s['train_x'] 
        train_label = h5s['train_label'] 
        test_x = h5s['test_x'] 
        test_label = h5s['test_label']
        sample_weights_train = h5s['sample_weights_train'] 
        sample_weights_test = h5s['sample_weights_test']
    logger.info('Features used in training are from sparse autoencoder')
train_x = pd.DataFrame(train_x)
train_label = pd.Series(train_label)
sample_weights_train = pd.Series(sample_weights_train)

labels = train_label.unique()
class_num = len(labels)
l_param = {'C': hp.uniform('C',0.05,10)}
if class_num <= 2:
    scoring = ['precision','recall','f1','roc_auc','neg_log_loss']
else:
    scoring = ['precision_macro','recall_macro','f1_macro','neg_log_loss']
    l_param['solver'] = hp.choice('solver',['lbfgs'])
    l_param['multi_class'] = hp.choice('multi_class',['multinomial'])
    print(l_param)
trial = Trials()


In [4]:
max_iter = 10
best = fmin(logistic_loss,l_param,algo=tpe.suggest,max_evals=max_iter,trials=trial) 
joblib.dump(best,model_dir+'LogisticRegression.pkl')

['/home/ec2-user/CpGPython/models/LogisticRegression.pkl']

In [5]:
b = joblib.load(model_dir+'LogisticRegression.pkl')

In [6]:
b

{'C': 6.383497398013961}

In [8]:
import sys
sys.executable

'/home/ec2-user/anaconda3/envs/py3.6/bin/python'

In [8]:
import shlex, subprocess

In [62]:
methods=['xgbooster']
feature_set = 'normal'
max_iter = 3
cv = 2
args={}
for method in methods:
    command_line = 'python {}code/HyperoptModels/{}_hyperopt.py -f {} -i {} -c {}'.format(home,method,feature_set,max_iter,cv)
    args[method] = shlex.split(command_line)



In [63]:
args

{'xgbooster': ['python',
  '/home/ec2-user/CpGPython/code/HyperoptModels/xgbooster_hyperopt.py',
  '-f',
  'normal',
  '-i',
  '3',
  '-c',
  '2']}

In [75]:
processes = {}
for method,arg in args.items():
    process = subprocess.Popen(arg)
    outs, errs = p.communicate()
    processes[method] = process


In [27]:
c = joblib.load(model_dir+'tensor_DNN.pkl')

In [30]:
from HyperoptModels import parallel_ensemble as pe

In [41]:
methods=['LogisticRegression','RandomForestClassifier','SVC','xgbooster','tensor_DNN']
es = pe.Ensemble(methods=methods)

In [42]:
es.fit(train_x,train_label,sample_weights_train)

{'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bytree': 1, 'gamma': 8.008342251635575, 'learning_rate': 0.1, 'max_delta_step': 0, 'max_depth': 9, 'min_child_weight': 1, 'missing': None, 'n_estimators': 624, 'n_jobs': 1, 'nthread': None, 'objective': 'binary:logistic', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 23.967889631668946, 'scale_pos_weight': 1, 'seed': None, 'silent': True, 'subsample': 1}
Tensor("IteratorGetNext:187", shape=(?,), dtype=float64)


Ensemble(home='/home/ec2-user/CpGPython/',
     methods=['LogisticRegression', 'RandomForestClassifier', 'SVC', 'xgbooster', 'tensor_DNN'])

In [43]:
score = es.score(test_x,test_label)

In [44]:
score

(0.26567417361647433,
 0.5714285714285714,
 0.5714285714285714,
 0.5714285714285714)